In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

In [2]:
!unzip -q lego_city_dataset_all_2019-03-16_12-52-20.zip

In [7]:
def get_steering(path):
    return (float(int(path[9:12])) - 50.0) / 50.0

def get_gaussian(mean, stdev, nbins):
    x = np.linspace(-1.0, 1.0, num=nbins)
    y = np.exp(-(x - mean)**2 / (stdev**2))
    return y

def get_bin(value, nbins):
    return np.digitize(value, np.linspace(-1.0, 1.0, num=nbins))

class SteeringDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, stdev=0.3, nbins=21):
        self.nbins = nbins
        self.stdev = stdev
        self.directory = directory
        self.image_paths = glob.glob(os.path.join(directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = PIL.Image.open(image_path)
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        steering = float(get_steering(os.path.basename(image_path)))
        #target = get_gaussian(steering, self.stdev, self.nbins)
        #target = get_bin(steering, self.nbins)
        
        return image, torch.tensor([steering]).float()
    
    
class MergedSteeringDataset(torch.utils.data.Dataset):
    
    def __init__(self, all_dir, task_dir, flipped=False):
        self.flipped = flipped
        self.all_dir = all_dir
        self.task_dir = task_dir
        self.all_image_paths = glob.glob(os.path.join(self.all_dir, '*.jpg'))
        self.task_image_paths = glob.glob(os.path.join(self.task_dir, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.all_image_paths) + len(self.task_image_paths)
    
    def __getitem__(self, idx):
        flip_all = False
        if idx < len(self.all_image_paths):
            image_path = self.all_image_paths[idx]
            
            if float(np.random.rand(1)) > 0.5:
                flip_all = True
        else:
            image_path = self.task_image_paths[idx - len(self.all_image_paths)]
        
        image = PIL.Image.open(image_path)
        steering = float(get_steering(os.path.basename(image_path)))
        
        if self.flipped:
            image = transforms.functional.hflip(image)
            steering = -steering
        
        if flip_all:
            image = transforms.functional.hflip(image)
            steering = -steering
            
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        #target = get_gaussian(steering, self.stdev, self.nbins)
        #target = get_bin(steering, self.nbins)
        
        return image, torch.tensor([steering]).float()

In [13]:
dataset = MergedSteeringDataset('dataset_all', 'dataset_forward', flipped=False)

test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

In [14]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 1)
device = torch.device('cuda')
model = model.to(device)

# train regression:

In [15]:

#optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(model.parameters())

In [16]:
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_steering_model_forward_merged.pth'
best_loss = 1e9


for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

0.791320, 6.682597
0.092692, 0.279749
0.051578, 0.067182
0.025356, 0.022788
0.023930, 0.017744
0.014398, 0.007435
0.020098, 0.006111
0.025016, 0.027790
0.019341, 0.014018
0.009976, 0.014225
0.024016, 0.011126
0.017086, 0.012246
0.008873, 0.010369
0.012482, 0.014907
0.014024, 0.008716
0.009774, 0.011792
0.010720, 0.006889
0.007576, 0.008988
0.005103, 0.005383
0.007039, 0.006632
0.011462, 0.016103
0.018346, 0.034427
0.013334, 0.016187
0.010101, 0.016813
0.012867, 0.005729
0.006141, 0.017357
0.015389, 0.008859
0.011402, 0.013094
0.013096, 0.006996
0.020413, 0.011713
0.013131, 0.006265
0.010761, 0.006492
0.004101, 0.006082
0.006606, 0.005042
0.010079, 0.015270
0.013460, 0.010705
0.015541, 0.008366
0.008351, 0.009810
0.007696, 0.005898
0.003434, 0.005218
0.004026, 0.003491
0.005082, 0.006590
0.006736, 0.014230
0.004962, 0.005807
0.006461, 0.004756
0.009112, 0.010780
0.013933, 0.013825
0.008843, 0.007743
0.003555, 0.007281
0.006331, 0.015034


In [16]:
torch.save(model.state_dict(), BEST_MODEL_PATH)

In [17]:
torch.save(model.state_dict(), 'best_steering_model_forward_merged_end.pth')